In [13]:
##  
import math
import numpy as np
import os
import pandas as pd

# Annotate 
def GetDist2Feats(x,H):
    print(H+".strand", x.shape)
    if x[H+".strand"] == "+":
        return x["str.start"] - x[H+'.start']
    elif x[H+".strand"] == "-":
        return -1*(x["str.start"] - x[H+'.stop'])
    else:
        return float(None)
    
def GetDist2Feat(strand, X, start,end):
   
    if str(strand) == "+":
        return X['str.start'].values[0] - start
    elif str(strand) == "-":
        return -1*(X['str.start'].values[0] - end)
    else:
        return float('NaN')
    
    
    
def AnnotateFeatures(X, ANOT):
    ANOT['start']=ANOT['gene.start'] ; ANOT['stop']=ANOT['gene.stop']
    chrom=['chr'+str(x) for x in range(1,23,1) ]+['X','Y']
    Annotated=[]
    
    for ch in chrom:
        Feature = X.loc[X['chrom']==ch]
        Names = ANOT.loc[ANOT['gene.chr']==ch].head(50)
        starts = list(Feature['start'])
        stops = list(Feature['stop'])
        Genes=[]
        i=-1
        for start in starts:
            i=i+1
            stop=stops[i]
            gene = Names.loc[(Names['start']<=start) & (Names['stop']>=stop)]['gene.id']
            print(start, stop, gene)
            try:
                Genes.append(list(gene)[0])
            except:
                Genes.append('nan')
        print(ch, ' AnnotateFeatures', Names.shape, len(Genes))      
        Feature['gene']= Genes  
        Annotated.append(Feature)
        break
    OUT = pd.concat(Annotated)
    return OUT

def ISINTHISFEATURE(X, feature):           #X=STRS
    start = X["start"].values[0]
    end = X["stop"].values[0]
    F_strs = feature[(feature["start"] <= start) & (feature["stop"] >= start)]
    if F_strs.shape[0] == 0:
        return 0,'nan', 'nan', 'nan'
    else:
        return 1,F_strs["start"].values[0], F_strs["stop"].values[0], F_strs["strand"].values[0]

    

ch = [i+1 for i in range(22)]#+['X','Y'] 
data=0; OUT=0;OUT1=0; p=0;i=0

In [16]:
DATA = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'#Merged_Best_causality.Table'#SUMMARY/Merged_Causality_best_tissue.table'
ANOT = '/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'
UTR3 = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/3UTR_table'
UTR5 = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/5UTR_table'
EXON = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/Exons_table'
ITRN = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/Introns_table'
CO_EX= '/storage/szfeupe/Runs/GTEx_estr/FEATURES/CodingExons_table'
MOTIF= '/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed'
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')


FEATCOL  = ['chrom', 'start', 'stop','description','score','strand', 'accession.code']


data0 = pd.read_csv(DATA, sep='\t')
data0['str.start']=data0['str.start'].astype(float)
data0 = data0[['gene','chrom','str.start', 'str.end', 'E.tissues']]
data = pd.merge(data0, Gene_table[["gene","chrom","gene.name"]], on=["chrom","gene"])


anot = pd.read_csv(ANOT,sep=',')                   ;  anot['gene']=anot['gene.id']

utr3 = pd.read_csv(UTR3, sep='\t', header=None)
utr3['accession.code']=[x.split('_ut')[0] for x in list(utr3[3])] ;  utr3.columns=FEATCOL

utr5 = pd.read_csv(UTR5, sep='\t', header=None)    
utr5['accession.code']=[x.split('_ut')[0] for x in list(utr5[3])] ;  utr5.columns=FEATCOL

exon = pd.read_csv(EXON , sep='\t', header=None)   
exon['accession.code']=[x.split('_ex')[0] for x in list(exon[3])] ;  exon.columns=FEATCOL

itrn = pd.read_csv(ITRN, sep='\t', header=None)    
itrn['accession.code']=[x.split('_in')[0] for x in list(itrn[3])] ;  itrn.columns=FEATCOL

coex = pd.read_csv(CO_EX, sep='\t', header=None)   
coex['accession.code']=[x.split('_cd')[0] for x in list(coex[3])] ;  coex.columns=FEATCOL

moti = pd.read_csv(MOTIF, sep='\t', header=None)
moti.columns=['chrom', 'start', 'stop', 'motif_len', 'motif']
moti['str.start']=moti['start'].astype(float)


In [17]:
#data['str.start']=data['best.str.start']
O = pd.merge(data, anot[["gene","gene.start","gene.stop","gene.strand"]], on=["gene"])

OUT1 = pd.merge(O, moti, on=["chrom","str.start"])
features = [utr3, utr5, exon, itrn]
headers = ['UTR3', 'UTR5', 'EXON', 'INTRON']
p=0
for feature in features:
    HEAD = headers[p]
    print(HEAD)
    OUT = OUT1
    frames = []
    print(p, OUT1.shape, HEAD)
    for C in ch:
        DF=[]
        starts = []
        ends = []
        Dist2Feat = []
        ds = OUT.loc[OUT['chrom']=='chr'+str(C)]
        ftr = feature.loc[feature['chrom']=='chr'+str(C)]
        ftr[HEAD+'.start']=ftr['start']
        ftr[HEAD+'.stop'] = ftr['stop']
        ftr[HEAD+'.strand'] = ftr['strand']
        for i in range(ds.shape[0]):
            X = ds.loc[[ds.index.values[i]]]
            output, output1, output2, strands = ISINTHISFEATURE(X, ftr)
            DF.append(output)
            starts.append(output1)
            ends.append(output2)
            if output==0:
                Dist2Feat.append('NaN')
            else:
                Dist2Feat.append(GetDist2Feat(strands, X, output1,output2))
        ds[HEAD]=DF
        ds[HEAD+'.start']=starts
        ds[HEAD+'.stop']=ends
        ds[HEAD+'.strand']=ftr['strand']
        ds['dist.to.'+HEAD]=Dist2Feat
        frames.append(ds)
    OUT1 = pd.concat(frames)
    p=p+1    
OUT1.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/SUMMARY/Annotated_LR_STRs.table', sep='\t', index=None)
OUT1

UTR3
0 (98671, 13) UTR3
1


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/p

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
UTR5
1 (98671, 18) UTR5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
EXON
2 (98671, 23) EXON
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
INTRON
3 (98671, 28) INTRON
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


,gene,chrom,str.start,str.end,E.tissues,gene.name,gene.start,gene.stop,gene.strand,start,...,EXON,EXON.start,EXON.stop,EXON.strand,dist.to.EXON,INTRON,INTRON.start,INTRON.stop,INTRON.strand,dist.to.INTRON
0,ENSG00000000457.9,chr1,169735051.0,169735063,0.0,SCYL3,169818772,169863408,-,169735051,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
1,ENSG00000000460.12,chr1,169735051.0,169735063,0.0,C1orf112,169631245,169823221,+,169735051,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
2,ENSG00000007908.11,chr1,169735051.0,169735063,0.0,SELE,169691781,169733846,-,169735051,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
3,ENSG00000171806.7,chr1,169735051.0,169735063,0.0,METTL18,169761670,169764107,-,169735051,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
4,ENSG00000188404.4,chr1,169735051.0,169735063,0.0,SELL,169659808,169680839,-,169735051,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
5,ENSG00000000457.9,chr1,169784101.0,169784116,0.0,SCYL3,169818772,169863408,-,169784101,...,0,nan,nan,+,NaN,1,169777070,169790819,+,7031
6,ENSG00000000460.12,chr1,169784101.0,169784116,0.0,C1orf112,169631245,169823221,+,169784101,...,0,nan,nan,+,NaN,1,169777070,169790819,+,7031
7,ENSG00000007908.11,chr1,169784101.0,169784116,0.0,SELE,169691781,169733846,-,169784101,...,0,nan,nan,+,NaN,1,169777070,169790819,+,7031
8,ENSG00000171806.7,chr1,169784101.0,169784116,0.0,METTL18,169761670,169764107,-,169784101,...,0,nan,nan,+,NaN,1,169777070,169790819,+,7031
9,ENSG00000000457.9,chr1,169895824.0,169895846,0.0,SCYL3,169818772,169863408,-,169895824,...,0,nan,nan,+,NaN,1,169890922,169923151,+,27327


In [21]:
p=0
eOUT = OUT1.loc[OUT1['E.tissues']>=1]
eshare = eOUT.loc[eOUT['E.tissues']>1]
print (OUT.shape, eOUT.shape, eshare.shape)
for feature in features:
    HEAD = headers[p]
    print('\n----------%s--------------'%HEAD)
    print(sum(OUT1[HEAD]),' STRs overlapping with ',HEAD)
    print(sum(eOUT[HEAD]),' eSTRs overlapping with ',HEAD)
    print(sum(eshare[HEAD]),' shared eSTRs overlapping with ',HEAD)
    p=p+1
    
    

(98671, 28) (9446, 33) (3663, 33)

----------UTR3--------------
1873  STRs overlapping with  UTR3
222  eSTRs overlapping with  UTR3
92  shared eSTRs overlapping with  UTR3

----------UTR5--------------
837  STRs overlapping with  UTR5
133  eSTRs overlapping with  UTR5
68  shared eSTRs overlapping with  UTR5

----------EXON--------------
2875  STRs overlapping with  EXON
377  eSTRs overlapping with  EXON
173  shared eSTRs overlapping with  EXON

----------INTRON--------------
55295  STRs overlapping with  INTRON
5653  eSTRs overlapping with  INTRON
2201  shared eSTRs overlapping with  INTRON


In [22]:
#data=OUT1
#genes=['ENSG00000164346.5', 'ENSG00000186470.9', 'ENSG00000204520.8', 'ENSG00000188234.9', 'ENSG00000149084.7', 'ENSG00000166435.11', 'ENSG00000166896.3', 'ENSG00000090857.9', 'ENSG00000141698.12', 'ENSG00000006282.15', 'ENSG00000174652.13']
#data.loc[data["gene"].isin(genes)][["gene","gene.name","motif","motif_len","chrom","str.start","best.score","best.tissue","EXON","INTRON","UTR3","UTR5"]]

OUT1.loc[OUT1['E.tissues']==17]

,gene,chrom,str.start,str.end,E.tissues,gene.name,gene.start,gene.stop,gene.strand,start,...,EXON,EXON.start,EXON.stop,EXON.strand,dist.to.EXON,INTRON,INTRON.start,INTRON.stop,INTRON.strand,dist.to.INTRON
1025,ENSG00000084072.12,chr1,40203639.0,40203665,17.0,PPIE,40157854,40229586,+,40203639,...,0,nan,nan,-,NaN,0,nan,nan,+,NaN
1926,ENSG00000236624.4,chr1,45950851.0,45950878,17.0,CCDC163P,45959538,45965724,-,45950851,...,0,nan,nan,+,NaN,1,45887518,45956523,+,5672
3400,ENSG00000117226.7,chr1,89465246.0,89465335,17.0,GBP3,89472349,89488577,-,89465246,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
4689,ENSG00000163041.5,chr1,226278030.0,226278068,17.0,H3F3A,226249578,226259702,+,226278030,...,0,nan,nan,+,NaN,0,nan,nan,+,NaN
58424,ENSG00000173209.18,chr2,61381024.0,61381055,17.0,AHSA2,61404553,61418338,+,61381024,...,0,nan,nan,NaN,NaN,1,61372331,61389976,+,8693
81667,ENSG00000163933.5,chr3,53128363.0,53128413,17.0,RFT1,53122499,53164478,-,53128363,...,0,nan,nan,NaN,NaN,1,53126634,53133396,NaN,5033
85694,ENSG00000164346.5,chr5,74013303.0,74013321,17.0,NSA2,74064756,74072737,+,74013303,...,0,nan,nan,NaN,NaN,1,74012498,74014115,NaN,805
86711,ENSG00000164308.12,chr5,96265112.0,96265195,17.0,ERAP2,96211643,96255420,+,96265112,...,0,nan,nan,NaN,NaN,0,nan,nan,NaN,NaN
91064,ENSG00000179344.12,chr6,32653747.0,32653758,17.0,HLA-DQB1,32627244,32636160,-,32653747,...,0,nan,nan,NaN,NaN,0,nan,nan,NaN,NaN
92568,ENSG00000121716.14,chr7,100055819.0,100055862,17.0,PILRB,99949799,99965356,+,100055819,...,0,nan,nan,NaN,NaN,1,100054608,100060985,NaN,5166


In [ ]:
#Making genecode table with features
HCOL=['gene','chrom','gene.name','feature.type','start','stop','G.strand','gene.type','database','score','genomic.phase']

annot = pd.read_csv('/storage/szfeupe/data/gencode.v19.annotation.gtf.gz', sep='\t', skiprows=5, header=None)
IDs= annot[8].str[1:-1].str.split('"', expand=True)
del annot[8]
annot.columns=['chrom','database','feature.type','start','stop','score','G.strand','genomic.phase']
annot['gene']=IDs[1]
annot['gene.name']=IDs[9]
annot['gene.type']=IDs[5]
annot = annot[HCOL]